<a href="https://colab.research.google.com/github/cristovamperes/mega-sena/blob/main/An%C3%A1lise_e_Previs%C3%A3o_Mega_Sena.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise e Previsão de Números da Mega-Sena: Explorando Padrões e Probabilidades

## Introdução

A Mega-Sena, a maior e mais popular loteria do Brasil, tem fascinado e estimulado a imaginação de milhões de pessoas. Com prêmios que podem mudar vidas, ela se tornou um tópico de interesse não apenas para aqueles que sonham em ganhar, mas também para entusiastas de estatísticas e análise de dados. Neste notebook, mergulharemos no mundo dos números da Mega-Sena, buscando padrões e tendências que possam nos ajudar a entender melhor este jogo intrigante.

##Objetivo do Projeto
O objetivo deste projeto é analisar os resultados históricos da Mega-Sena e explorar se existem padrões ou tendências nos números sorteados ao longo dos anos. Utilizaremos técnicas de análise de dados e programação para extrair insights desses resultados. A partir disso, tentaremos desenvolver um método para sugerir combinações de números que podem ser consideradas para apostas futuras. É importante destacar que, apesar de nosso foco em padrões e probabilidades, a Mega-Sena é um jogo de sorte, e não há garantias de sucesso.

#Metodologia
Para realizar nossa análise, utilizaremos dados históricos dos sorteios da Mega-Sena. Esses dados incluem as datas dos sorteios e os números sorteados. Através de técnicas de programação em Python e o uso de bibliotecas como requests para aquisição de dados e pandas para manipulação e análise, processaremos e analisaremos essas informações.

##Estrutura do Notebook
Este notebook está organizado da seguinte forma:

*   **Coleta de Dados:** Mostraremos como acessar e baixar os resultados históricos da Mega-Sena

*   **Análise Exploratória dos Dados:** Analisaremos os dados coletados para identificar quaisquer padrões ou tendências nos números sorteados.

*   **Desenvolvimento do Modelo de Previsão:** Com base na nossa análise, desenvolveremos um modelo para prever números que poderiam ser sorteados em futuros jogos.

*   Conclusão: Discutiremos os insights obtidos, as limitações do nosso estudo e as considerações finais sobre o uso de análise de dados em jogos de loteria.

Vamos começar!

# Célula 1: Coleta de Dados dos Resultados da Mega-Sena

## Explicação do Código

1.   **Importando as Bibliotecas:** Utilizamos requests para fazer a requisição dos dados e pandas para manipular os dados em formato de DataFrame.

2.   **Requisição de Dados:** A função requests.post é usada para enviar uma requisição POST para o site com os resultados da Mega-Sena. Os parâmetros enviados na requisição especificam o formato e o tipo de dados que queremos.

3.   **Salvando e Lendo o Arquivo Excel:** O conteúdo recebido é salvo em um arquivo Excel local. Em seguida, utilizamos pandas para ler este arquivo, especificando as linhas e colunas que devem ser ignoradas para estruturar corretamente os dados.

4.   **Preparação dos Dados:** Por fim, invertemos a ordem dos dados para que os sorteios mais antigos sejam listados primeiro, facilitando análises temporais subsequentes.

Com esses dados em mãos, estamos prontos para avançar para a próxima etapa de análise exploratória e modelagem preditiva.

In [1]:
import requests
import pandas as pd

# URL do site que contém os resultados da Mega-Sena em formato Excel
url = 'https://asloterias.com.br/download_excel.php'

# Parâmetros para a requisição POST
data = {"l": "ms", "t": "t", "o": "s", "f1": "", "f2": ""}

# Realizando a requisição e recebendo o arquivo
req_file = requests.post(url, data)

# Nome do arquivo onde os dados serão salvos
file_name = "todos_resultados_megasena.xlsx"

# Salvando o conteúdo recebido em um arquivo Excel
with open(file_name, 'wb') as f:
    f.write(req_file.content)

# Lendo os dados do arquivo Excel com Pandas
df = pd.read_excel(io=file_name, sheet_name='mega_sena_www.asloterias.com.br', skiprows=[0,1,2,4,5], header=[1], engine='openpyxl')

# Invertendo a ordem dos dados para que os sorteios mais antigos apareçam primeiro
df = df.iloc[::-1]

# Célula 2: Desenvolvimento da Função de Previsão

Nesta seção, vamos criar uma função chamada *predict_next_numbers* que analisa os resultados históricos e sugere números para futuros sorteios da Mega-Sena. Esta função utiliza a análise de frequência dos números sorteados e seleciona números com base em sua ocorrência nos últimos sorteios.

## Explicação do Código


1.   **Definição da Função:** predict_next_numbers recebe um DataFrame df contendo os resultados da Mega-Sena e um número sorteios que indica quantos sorteios recentes considerar na análise.

2.   **Análise de Frequência:** Para cada coluna que representa uma bola sorteada, contamos a frequência de cada número nos últimos sorteios usando a classe Counter.

3.   **Agrupamento por Frequência:** Os números são agrupados com base na sua frequência de ocorrência. Em seguida, esses grupos são ordenados.

4.   **Seleção dos Números:** A função então escolhe números aleatoriamente, começando pelo grupo de menor frequência e evitando números já selecionados. Esse processo ajuda a diversificar os números escolhidos.

5.   Retorno dos Números Previstos: A função retorna os números previstos em ordem crescente.


Com essa função, estamos preparados para gerar sugestões de números para futuros jogos. A lógica busca equilibrar a seleção de números com base em sua frequência histórica, considerando que os números com menos ocorrências, tem maior probabilidade de serem sorteados, assim como os resultados de milhares de lançamentos de uma moeda terminará com 50% para cada lado.

In [2]:
import pandas as pd
import random
from collections import Counter

def predict_next_numbers(df, sorteios):
    total_sorteios = sorteios
    predicted_numbers = set()  # Conjunto para armazenar os números previstos

    for i, bola in enumerate(df.filter(regex='bola').columns):
        numeros_sorteados = df[bola].head(total_sorteios).squeeze()

        # Contagem das ocorrências de cada número
        counter = Counter(numeros_sorteados)

        # Agrupamento dos números por sua frequência
        freq_groups = {}
        for num, freq in counter.items():
            if freq not in freq_groups:
                freq_groups[freq] = []
            freq_groups[freq].append(num)

        # Ordenação dos grupos de frequência
        sorted_freqs = sorted(freq_groups.keys())

        # Seleção do próximo número, iniciando pelo grupo de menor frequência
        next_number = None
        for freq in sorted_freqs:
            potential_numbers = [num for num in freq_groups[freq] if num not in predicted_numbers]
            if potential_numbers:
                next_number = random.choice(potential_numbers)
                break

        if next_number is None:
            print("Cenário inesperado: não foi possível encontrar um número único.")
            continue

        predicted_numbers.add(next_number)

    return sorted(predicted_numbers)


#Célula 3: Geração de Sugestões de Jogos

Nesta parte final da nossa análise, utilizaremos a função *predict_next_numbers *para gerar sugestões de jogos da Mega-Sena. O usuário poderá especificar quantas sugestões de jogos deseja gerar. Cada sugestão será uma combinação única de números baseada nos resultados dos sorteios anteriores.

#Explicação do Código

1.   **Definindo o Número de Sugestões:** O código começa perguntando ao usuário quantas sugestões de jogos ele deseja gerar. Isso é feito usando a função input.

2.   **Geração de Sugestões:** Utilizamos um loop while para gerar o número desejado de sugestões. Em cada iteração, chamamos a função *predict_next_numbers*, passando o DataFrame com os resultados da Mega-Sena e o número de sorteios a considerar.

3.   **Evitando Repetições:** Verificamos se a sugestão gerada já existe na lista de jogos. Se não, ela é adicionada à lista. Isso garante que todas as sugestões sejam únicas.

4.   **Exibindo as Sugestões:** Após a geração das sugestões, cada jogo sugerido é exibido na tela. Isso é feito em um loop for, que percorre a lista de jogos sugeridos e os imprime formatadamente.

Com este código, oferecemos ao usuário uma maneira interativa e interessante de explorar possíveis combinações de números para jogar na Mega-Sena, com base em uma análise estatística dos resultados anteriores.

In [4]:
suggestions_to_generate = int(input("Quantas sugestões quer gerar? "))
ultimo = 0
jogos = []

while len(jogos) < suggestions_to_generate:
    sorteios = (len(df) - ultimo)
    sugestao_jogo = predict_next_numbers(df, sorteios)

    if sugestao_jogo not in jogos:
        jogos.append(sugestao_jogo)
        print(f"{len(jogos)} jogo(s) sugerido(s)")
    ultimo += 1

print("Sugestões finalizadas\n")

for x in range(len(jogos)):
    print(f"Jogo {x+1}: {jogos[x]}")


Quantas sugestões quer gerar? 10
1 jogo(s) sugerido(s)
2 jogo(s) sugerido(s)
3 jogo(s) sugerido(s)
4 jogo(s) sugerido(s)
5 jogo(s) sugerido(s)
6 jogo(s) sugerido(s)
7 jogo(s) sugerido(s)
8 jogo(s) sugerido(s)
9 jogo(s) sugerido(s)
10 jogo(s) sugerido(s)
Sugestões finalizadas

Jogo 1: [3, 15, 21, 26, 55, 60]
Jogo 2: [3, 21, 26, 40, 55, 60]
Jogo 3: [3, 18, 21, 26, 55, 60]
Jogo 4: [3, 15, 21, 22, 55, 60]
Jogo 5: [3, 21, 22, 40, 55, 60]
Jogo 6: [3, 18, 21, 22, 55, 60]
Jogo 7: [3, 21, 22, 48, 55, 60]
Jogo 8: [3, 21, 26, 48, 55, 60]
Jogo 9: [3, 18, 21, 26, 38, 55]
Jogo 10: [3, 15, 21, 22, 38, 55]


#Conclusão e Reflexões Finais

##Resumo dos Insights
Neste notebook, exploramos um método interessante de análise dos resultados históricos da Mega-Sena, com o objetivo de identificar padrões e sugerir possíveis números para futuros sorteios. Utilizamos a programação em Python para coletar dados, analisar a frequência dos números sorteados e, com base nessa análise, sugerir combinações de números.

##Limitações do Estudo
É importante reconhecer as limitações do nosso estudo:


*   **Aleatoriedade dos Sorteios:** Os sorteios da Mega-Sena são eventos aleatórios. Portanto, qualquer padrão identificado nos dados históricos não garante resultados futuros.

*  **Tamanho da Amostra:** Embora tenhamos uma quantidade significativa de dados históricos, a natureza aleatória dos sorteios significa que grandes variações são possíveis e comuns.

*   **Simplificação do Modelo:** O modelo que desenvolvemos é baseado em frequência de números e não leva em conta outras técnicas estatísticas ou probabilísticas mais complexas que poderiam refinar as previsões.

##Reflexões Finais
Este projeto demonstra o poder da análise de dados e programação para explorar e entender melhor jogos de loteria como a Mega-Sena. Enquanto a previsão de resultados futuros em jogos de sorte é inerentemente incerta, a análise de dados pode oferecer insights interessantes e uma nova perspectiva sobre os padrões dos números sorteados.

#Encorajamento para Exploração Futura
Além do método explorado neste notebook, eu já experimentei com diversas outras técnicas de análise de dados e modelagem estatística para prever os números da Mega-Sena, incluindo:

*   **Modelo ARIMA:** ARIMA, que significa "AutoRegressive Integrated Moving Average", é um modelo estatístico usado para prever séries temporais. Ele combina técnicas autoregressivas, diferenciação para tornar a série temporal estacionária, e médias móveis para capturar a dependência temporal nos dados.

*   **Regressão Linear:** Uma técnica de modelagem estatística que tenta prever o valor de uma variável dependente com base nas relações lineares entre essa variável e uma ou mais variáveis independentes. É uma das formas mais básicas e amplamente utilizadas de análise preditiva.

*   **Modelo LSTM (Long Short-Term Memory):** Um tipo de rede neural recorrente (RNN) que é particularmente eficaz em aprender dependências de longo prazo em dados sequenciais. LSTMs são frequentemente usadas em problemas de séries temporais, como previsão do mercado de ações ou análise de tendências.

Além dessas técnicas, outra abordagem interessante é pedir para ferramentas de IA como o ChatGPT analisar os resultados e sugerir números com base nos dados disponíveis. Embora essa abordagem possa não se basear em modelos estatísticos tradicionais, ela oferece uma perspectiva única e criativa sobre a previsão de resultados.

Encorajo você a explorar essas e outras técnicas em seus próprios projetos. Cada método tem suas vantagens e peculiaridades, e a experimentação com diferentes abordagens pode levar a insights valiosos e novas maneiras de pensar sobre os dados. Lembre-se, a análise de dados é um campo vasto e em constante evolução, e sempre há algo novo para aprender e experimentar!